### Restart and Run All Cells

### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [3]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

# Define formatting dictionary for all columns
format_dict = {
    'qty': '{:,.0f}',          # Comma-separated, no decimals (10,000)
    'days': '{:,}',     
    'sell_price': '{:.2f}',     # 2 decimals (2.82)
    'price': '{:.2f}',          # 2 decimals (1.87)
    'high': '{:.2f}',          # 2 decimals (1.87)
    'low': '{:.2f}',          # 2 decimals (1.87)    
    'target': '{:.2f}',          # 2 decimals (1.87)    
    'buy_price': '{:.2f}',          # 2 decimals (1.87)    
    'difa': '{:,.2f}',          # Comma-separated, 2 decimals (-9,500.00)
    'diff': '{:.2f}',          # 2 decimals (1.87)
    'sell_amt': '{:,.2f}',          # 2 decimals (1.87)    
    'buy_amt': '{:,.2f}',          # Comma-separated, 2 decimals (-9,500.00)    
    'amount': '{:,.2f}',          # Comma-separated, 2 decimals (-9,500.00)     
    'gross': '{:,.2f}',          # 2 decimals (1.87)    
    'pct': '{:,.2f}%',          # Comma-separated, 2 decimals (-9,500.00)    
    '%change': '{:.2f}%',        # 2 decimals (-1.58)
    '%profit': '{:.2f}%',        # 2 decimals (-33.69)
    '%decline': '{:.2f}%',        # 2 decimals (-33.69)    
    'sell_date': '{:%Y-%m-%d}'  # Date without time (2025-03-03)
}

pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-08-15 2025-08-14


In [4]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-08-15 21:07:45


In [5]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
dat_path = os.path.join(base_path, "Data")
print(f"Data path (dat_path): {dat_path}") 
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [6]:
prices = pd.read_csv(input_file)
prices.dtypes

name         object
date         object
price       float64
change      float64
%change     float64
open        float64
high        float64
low         float64
volume        int64
value(m)    float64
dtype: object

### Purchase recently sold stocks if price lower than last sold

In [8]:
#Select name from portfolio
sql = """SELECT name FROM buy"""
portfolio = pd.read_sql(sql, const)
names = portfolio.name
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'3BBIF', 'AH', 'AIMIRT', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [9]:
#
sql = """
SELECT T.name, S.date AS sell_date, S.price AS sell_price, B.price AS buy_price, B.date AS buy_date, B.qty
FROM sells S
JOIN buys B
ON S.buy_id = B.id 
JOIN stocks T
ON T.id = B.stock_id 
WHERE T.name IN (%s)
ORDER BY T.name, S.date DESC"""
sql = sql % (in_p)
sells = pd.read_sql(sql, conpf)
sells.shape

(231, 6)

In [10]:
sells.dtypes

name           object
sell_date      object
sell_price    float64
buy_price     float64
buy_date       object
qty             int64
dtype: object

In [11]:
# Convert dates to datetime if they're not already
sells['sell_date'] = pd.to_datetime(sells['sell_date'])

# Get the latest record for each name
latest_records = sells.sort_values('sell_date', ascending=False).drop_duplicates('name')
latest_records.shape

(26, 6)

In [12]:
colu = 'name sell_date qty sell_price price low target %decline'.split()
sell_data   = pd.merge(latest_records, prices, on='name', how='inner')
sell_data['diff'] = sell_data['price'] - sell_data['sell_price']
sell_data['%decline'] = round(sell_data['diff'] / (sell_data['sell_price']) * 100,2)
#sell_data['sell_date'] = pd.to_datetime(sell_data['sell_date'])
sell_data['target'] = round(sell_data['sell_price'] * 0.9,2)
#sell_data['amount'] = round(sell_data['target'] * sell_data['qty'])

In [13]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_price'] >= sell_data['price']),
    colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)

# Format display (if format_dict is defined)
filtered.style.format(format_dict)

,name,sell_date,qty,sell_price,price,low,target,%decline
17,SINGER,2023-01-09,"3,600",29.25,5.30,5.20,26.32,-81.88%
25,PTG,2021-03-02,"6,000",19.90,7.00,6.80,17.91,-64.82%
22,SENA,2021-12-17,"75,000",4.08,1.79,1.77,3.67,-56.13%
16,PTTGC,2023-01-17,"3,000",50.75,22.70,22.60,45.68,-55.27%
23,TOA,2021-09-21,"3,000",33.00,14.90,14.60,29.70,-54.85%
19,SCC,2022-11-07,300,340.00,201.00,201.00,306.00,-40.88%
14,BCH,2024-01-04,"2,000",22.90,13.60,13.50,20.61,-40.61%
15,STA,2024-01-03,"2,500",16.90,11.90,11.70,15.21,-29.59%
18,SYNEX,2022-12-30,"3,000",16.50,11.70,11.50,14.85,-29.09%
21,AIMIRT,2022-04-18,"10,000",12.70,9.85,9.80,11.43,-22.44%


In [14]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_date'].dt.year == 2025) & 
    (sell_data['sell_price'] >= sell_data['price']),
    colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)

# Format display (if format_dict is defined)
filtered.style.format(format_dict)

,name,sell_date,qty,sell_price,price,low,target,%decline
3,JMART,2025-08-07,"3,200",9.40,8.25,8.05,8.46,-12.23%
1,RCL,2025-08-08,"3,000",31.00,28.25,28.00,27.90,-8.87%
6,IVL,2025-07-30,"1,800",22.90,21.60,21.50,20.61,-5.68%
0,GVREIT,2025-08-08,"6,000",6.70,6.50,6.45,6.03,-2.99%
4,NER,2025-08-06,"5,000",4.52,4.48,4.44,4.07,-0.88%
5,JMT,2025-08-06,"2,800",11.60,11.50,11.30,10.44,-0.86%


In [15]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
ruby_path = os.path.join(base_path, "Ruby")
print(f"Ruby path (ruby_path): {ruby_path}") 

Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby


In [16]:
file_name = "watch-list-buys.xlsx"  
output_file = os.path.join(ruby_path, file_name)

# Apply formatting to the DataFrame (not Styler)
filtered_fmt = filtered.copy()
filtered_fmt['sell_date'] = filtered_fmt['sell_date'].dt.strftime('%Y-%m-%d')  # Format date
filtered_fmt['sell_price'] = filtered_fmt['sell_price'].map('{:,.2f}'.format)  # Format currency
filtered_fmt['price'] = filtered_fmt['price'].map('{:.2f}'.format)  # Format currency
filtered_fmt['low'] = filtered_fmt['low'].map('{:.2f}'.format)  # Format currency
filtered_fmt['target'] = filtered_fmt['target'].map('{:.2f}'.format)  # Format currency
filtered_fmt['%decline'] = filtered_fmt['%decline'].map('{:.2f}%'.format)  # Format percentage
filtered_fmt['qty'] = filtered_fmt['qty'].map('{:,}'.format)  # Format qty with commas
#filtered_fmt['amount'] = filtered_fmt['amount'].map('{:,.2f}'.format)  # Format currency

# Export to Excel
filtered_fmt.sort_values('%decline', ascending=True).to_excel(
    output_file, 
    header=True, 
    index=False
)

### Profit per Month

In [18]:
year = 2025
month = 7
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 7
ORDER BY sells.date DESC, name


In [19]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,IVL,2025,7,22.90,20.00,2.90,"1,800","41,220.00","36,000.00","5,220.00",14.50%,51
1,WHAIR,2025,7,5.50,4.70,0.80,"10,000","55,000.00","47,000.00","8,000.00",17.02%,75
2,KCE,2025,7,22.90,87.00,-64.10,"1,000","22,900.00","87,000.00","-64,100.00",-73.68%,"1,391"
3,KCE,2025,7,22.40,87.00,-64.60,"1,000","22,400.00","87,000.00","-64,600.00",-74.25%,"1,386"
4,KCE,2025,7,22.20,87.00,-64.80,"1,000","22,200.00","87,000.00","-64,800.00",-74.48%,"1,312"
5,KCE,2025,7,21.70,87.00,-65.30,"1,000","21,700.00","87,000.00","-65,300.00",-75.06%,"1,310"
6,JMT,2025,7,10.20,9.00,1.20,"2,500","25,500.00","22,500.00","3,000.00",13.33%,29
7,KCE,2025,7,21.00,81.00,-60.00,"1,000","21,000.00","81,000.00","-60,000.00",-74.07%,"1,262"
8,KCE,2025,7,19.60,81.00,-61.40,"1,000","19,600.00","81,000.00","-61,400.00",-75.80%,"1,261"


In [20]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: -363,980.00


In [21]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-08-15 21:07:45
